<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-1-NLP/blob/master/OK_Copy_Spacy_RFC_RFSearchCV_0_9split_whiskey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

### Load Competition Data

In [0]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('https://raw.githubusercontent.com/bs3537/DS-Unit-4-Sprint-1-NLP/master/module3-document-classification/train.csv')
#test = pd.read_csv('test.csv')

In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [0]:
train['text'] = train['description'].str.strip('\n')

In [0]:
train2 = train.drop(columns=['description', 'id'])

In [0]:
train3 = train2.rename(columns={"ratingCategory": "label"})

In [6]:
train3.head()

,label,text
0,1,"Sometimes, when whisky is batched, a few lefto..."
1,0,An uncommon exclusive bottling of a 6 year old...
2,1,This release is a port version of Amrut’s Inte...
3,1,This 41 year old single cask was aged in a she...
4,1,"Quite herbal on the nose, with aromas of dried..."


In [7]:
from sklearn.model_selection import train_test_split

df_trn, df_val = train_test_split(train3, stratify = train3['label'], test_size = 0.10, random_state=42)

df_trn.shape, df_val.shape

((3678, 2), (409, 2))

In [8]:
#Vectorize text using spacy
pip install -U spacy[cuda92]

Requirement already up-to-date: spacy[cuda92] in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [9]:
!python3 -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import spacy

spacy.prefer_gpu()
nlp = spacy.load('en_core_web_lg')

In [0]:
doc = nlp("NLP is awesome!")

In [0]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [0]:
import numpy as np
X = df_trn['text']
Y = df_trn['label']
X_train_spacy = get_word_vectors(X)
X_train = np.array(X_train_spacy)

In [0]:
# Import Statements
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

### Define Pipeline Components

In [15]:
%time
pipeline = make_pipeline(RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_features='auto'))

#define parameter distributions for RandomizedSearchCV
param_distributions = { 
    'randomforestclassifier__n_estimators': randint(100, 1000), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, 25, 30, None], 
}

rf_search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=20, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1,
    refit=True,
    verbose=3
)

rf_search.fit(X_train, Y);

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs
Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 10.2min finished


In [21]:
print('Best hyperparameters', rf_search.best_params_)
print('Accuracy', rf_search.best_score_)
print('Best estimator', rf_search.best_estimator_)

Best hyperparameters {'randomforestclassifier__max_depth': None, 'randomforestclassifier__n_estimators': 402}
Accuracy 0.7357264862466726
Best estimator Pipeline(memory=None,
         steps=[('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=402, n_jobs=-1,
                                        oob_score=False, random_state=42,
                                        verbose=0, w

In [0]:
#make predictions on test set
pipeline = rf_search.best_estimator_

In [0]:
#Predict on val data

X_val = df_val['text']
y_val = df_val['label']
X_val_spacy = get_word_vectors(X_val)
X_val = np.array(X_val_spacy)

In [0]:
y_pred = pipeline.predict(X_val)


In [26]:
from sklearn.metrics import accuracy_score
val_accuracy = accuracy_score(y_val, y_pred)
print("RandomSearchCV Validation Accuracy=", val_accuracy)

RandomSearchCV Validation Accuracy= 0.7506112469437652


In [0]:
#Slight increase in Val. accuracy on increasing the train size from 80/20 split to 90/10 split

#Val. accuracy is in the same range as Keras Deep FF NN but the results are more easily applicable on a test dataset for predictions.

#Val. accuracy also in same range as TPOTClassifier but faster run by RFClassifier _RandomSearch CV using 90/10 split

